In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
import pylab
import pickle
import copy
import processing
import matplotlib.animation as animation
%matplotlib inline

WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-07-04'
RawDataPath=WORKSPACE_PATH+'/raw/'
StudyDataPath=WORKSPACE_PATH+'/study/'
GlaphDataPath=WORKSPACE_PATH+'/glaph/'
GlaphDataFftPath=WORKSPACE_PATH+'/fft/'
WindowDataPath=WORKSPACE_PATH+'/window/'
StudyDataModelPicPath=StudyDataPath+'modelPic/'

In [2]:
from keras.models import Sequential
from keras.datasets import cifar10
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils.visualize_util import model_to_dot, plot
from keras import backend as K
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 4007)


In [3]:
"""
    encordData,decordData
    firstNum:
    goalNum:
    commonLabel:共通名前
"""
def MakeGlaph(encordData,decordData,firstNum,goalNum,commonLabel):
    try:
        eRow, eCol=encordData.shape
        dRow, dCol=decordData.shape
    except ValueError:
        eRow=1
        dRow=1
        eCol = len(encordData)
        dCol = len(decordData)
        
    if (eRow!=dRow) | (eCol!=dCol):
        print 'check encordData & decordData'
        return -1
        
    start = firstNum
    goal=goalNum
    print str(start)+' is start '+str(goal)+' is goal'
    dataE = encordData.reshape(eRow*eCol,1)[start:goal]
    dataD = decordData.reshape(dRow*dCol,1)[start:goal]
    plt.figure(figsize=(160, 90))
    plt.plot(range(0,len(dataE)),dataE,'--r',label="Encoded")
    plt.plot(range(0,len(dataD)),dataD,'b',label="Decoded")
    plt.legend(loc='best')
    plt.title(str(start)+"-"+str(goal))
    plt.savefig(commonLabel+'-'+str(start)+"-"+str(goal)+'-'+'glaph')
    plt.close()

辞書型の変数を読み込む

In [4]:
dic1=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor1")
dic2=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor2")
dic3=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor3")
dic4=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor4")
dic5=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor5")
dic6=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor6")
dic7=processing.LoadDicDataFromFileNPZ(RawDataPath+"MemSensor7")

windowにしたものを読み込む

In [5]:
def CalcFFT(self,samp,overlap=0.5):
        start = time.time()
        fft_data = np.array([])
        
        for n,f in zip( self._columns,self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            while True:
                try:
                    d=sw.next()
                    fft_data = np.append(fft_data, fftpack.fft(d)[1:(samp/2)+1] )   # fftの直流成分を除くsample/2の
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    fft_data = fft_data.reshape(len(fft_data)/(samp/2),(samp/2) )
                    self._FFTData[f] = fft_data
                    np.savez(p_path+'fft/'+self.ClassName+'_'+str(f)+'_'+str(samp),self._FFTData[f])
                    break   
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

In [6]:
encord=processing.LoadDicDataFromFileNPZ('window/sensor7_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/sensor7_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:5])
        fftDataD=np.append(fftDataD, fftpack.fft(encord[i])[1:5])

fftDataE = fftDataE.reshape(len(encord),4)
fftDataD = fftDataD.reshape(len(encord),4)

In [7]:
print fftDataE.shape
print fftDataD.shape

(15921, 4)
(15921, 4)


In [8]:
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

In [9]:
PowerE.shape

(15921, 4)

In [10]:
Sensor ='sensor1'
encord=processing.LoadDicDataFromFileNPZ('window/'+Sensor+'_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/'+Sensor+'_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:len(encord[i])/2+1])
        fftDataD=np.append(fftDataD, fftpack.fft(decord[i])[1:len(encord[i])/2+1])

fftDataE = fftDataE.reshape(len(encord),len(encord[0])/2)
fftDataD = fftDataD.reshape(len(encord),len(encord[0])/2)

for i in range(4):
    MakeGlaph(np.imag(fftDataE[:,i]), np.imag(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-image-AccX')
    
for i in range(4):
    MakeGlaph(np.real(fftDataE[:,i]), np.real(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-real-AccX')
    
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

for i in range(4):
    MakeGlaph(PowerE[:,i], PowerD[:,i],0,len(PowerE),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-power-AccX')

0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal


In [11]:
Sensor ='sensor2'
encord=processing.LoadDicDataFromFileNPZ('window/'+Sensor+'_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/'+Sensor+'_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:len(encord[i])/2+1])
        fftDataD=np.append(fftDataD, fftpack.fft(decord[i])[1:len(encord[i])/2+1])

fftDataE = fftDataE.reshape(len(encord),len(encord[0])/2)
fftDataD = fftDataD.reshape(len(encord),len(encord[0])/2)

for i in range(4):
    MakeGlaph(np.imag(fftDataE[:,i]), np.imag(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-image-AccX')
    
for i in range(4):
    MakeGlaph(np.real(fftDataE[:,i]), np.real(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-real-AccX')
    
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

for i in range(4):
    MakeGlaph(PowerE[:,i], PowerD[:,i],0,len(PowerE),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-power-AccX')

0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal


In [12]:
Sensor ='sensor3'
encord=processing.LoadDicDataFromFileNPZ('window/'+Sensor+'_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/'+Sensor+'_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:len(encord[i])/2+1])
        fftDataD=np.append(fftDataD, fftpack.fft(decord[i])[1:len(encord[i])/2+1])

fftDataE = fftDataE.reshape(len(encord),len(encord[0])/2)
fftDataD = fftDataD.reshape(len(encord),len(encord[0])/2)

for i in range(4):
    MakeGlaph(np.imag(fftDataE[:,i]), np.imag(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-image-AccX')
    
for i in range(4):
    MakeGlaph(np.real(fftDataE[:,i]), np.real(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-real-AccX')
    
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

for i in range(4):
    MakeGlaph(PowerE[:,i], PowerD[:,i],0,len(PowerE),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-power-AccX')

0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal


In [13]:
Sensor ='sensor4'
encord=processing.LoadDicDataFromFileNPZ('window/'+Sensor+'_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/'+Sensor+'_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:len(encord[i])/2+1])
        fftDataD=np.append(fftDataD, fftpack.fft(decord[i])[1:len(encord[i])/2+1])

fftDataE = fftDataE.reshape(len(encord),len(encord[0])/2)
fftDataD = fftDataD.reshape(len(encord),len(encord[0])/2)

for i in range(4):
    MakeGlaph(np.imag(fftDataE[:,i]), np.imag(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-image-AccX')
    
for i in range(4):
    MakeGlaph(np.real(fftDataE[:,i]), np.real(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-real-AccX')
    
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

for i in range(4):
    MakeGlaph(PowerE[:,i], PowerD[:,i],0,len(PowerE),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-power-AccX')

0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal


In [14]:
Sensor ='sensor5'
encord=processing.LoadDicDataFromFileNPZ('window/'+Sensor+'_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/'+Sensor+'_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:len(encord[i])/2+1])
        fftDataD=np.append(fftDataD, fftpack.fft(decord[i])[1:len(encord[i])/2+1])

fftDataE = fftDataE.reshape(len(encord),len(encord[0])/2)
fftDataD = fftDataD.reshape(len(encord),len(encord[0])/2)

for i in range(4):
    MakeGlaph(np.imag(fftDataE[:,i]), np.imag(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-image-AccX')
    
for i in range(4):
    MakeGlaph(np.real(fftDataE[:,i]), np.real(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-real-AccX')
    
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

for i in range(4):
    MakeGlaph(PowerE[:,i], PowerD[:,i],0,len(PowerE),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-power-AccX')

0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal


In [15]:
Sensor ='sensor6'
encord=processing.LoadDicDataFromFileNPZ('window/'+Sensor+'_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/'+Sensor+'_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:len(encord[i])/2+1])
        fftDataD=np.append(fftDataD, fftpack.fft(decord[i])[1:len(encord[i])/2+1])

fftDataE = fftDataE.reshape(len(encord),len(encord[0])/2)
fftDataD = fftDataD.reshape(len(encord),len(encord[0])/2)

for i in range(4):
    MakeGlaph(np.imag(fftDataE[:,i]), np.imag(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-image-AccX')
    
for i in range(4):
    MakeGlaph(np.real(fftDataE[:,i]), np.real(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-real-AccX')
    
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

for i in range(4):
    MakeGlaph(PowerE[:,i], PowerD[:,i],0,len(PowerE),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-power-AccX')

0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal


In [16]:
Sensor ='sensor7'
encord=processing.LoadDicDataFromFileNPZ('window/'+Sensor+'_AccX_train.npz')
decord=processing.LoadDicDataFromFileNPZ('study/'+Sensor+'_AccX_decoded.npz')

fftDataE = np.array([])
fftDataD = np.array([])

for i in range(len(encord)):
        fftDataE=np.append(fftDataE, fftpack.fft(encord[i])[1:len(encord[i])/2+1])
        fftDataD=np.append(fftDataD, fftpack.fft(decord[i])[1:len(encord[i])/2+1])

fftDataE = fftDataE.reshape(len(encord),len(encord[0])/2)
fftDataD = fftDataD.reshape(len(encord),len(encord[0])/2)

for i in range(4):
    MakeGlaph(np.imag(fftDataE[:,i]), np.imag(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-image-AccX')
    
for i in range(4):
    MakeGlaph(np.real(fftDataE[:,i]), np.real(fftDataD[:,i]),
              0,len(fftDataD),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-real-AccX')
    
PowerE = np.imag(fftDataE)**2+np.real(fftDataE)**2
PowerD = np.imag(fftDataD)**2+np.real(fftDataD)**2

for i in range(4):
    MakeGlaph(PowerE[:,i], PowerD[:,i],0,len(PowerE),GlaphDataFftPath+Sensor+'-'+str(i)+'-fft-power-AccX')

0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
0 is start 15921 is goal
